In [87]:
import pandas as pd
import re
import uuid
import json
import requests

from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [53]:
df = pd.read_csv('/SUPA Triple Rumble Season 1 Sign Ups - Team Responses.csv')

In [94]:
print(df.head(1).iloc[:, 8])

0          https://www.twitch.tv/lucazzyt
1      https://www.twitch.tv/quiet_kid_15
2          https://www.twitch.tv/datsurb1
3             https://www.twitch.tv/ej_ca
4           https://www.twitch.tv/bytesy_
5               https://twitch.tv/noam15A
6            https://www.twitch.tv/mr_bjo
7       https://www.twitch.tv/blobby56789
8           https://www.twitch.tv/asmuund
9       https://twitch.tv/Stud3nt_4thlete
10     https://www.twitch.tv/thanighthawk
11         https://www.twitch.tv/kryptecc
12          https://www.twitch.tv/tseska_
13       https://www.twitch.tv/generaldum
14    https://www.twitch.tv/nolanimations
15               https://twitch.tv/rfcaps
16         https://www.twitch.tv/oriixenn
17           https://twitch.tv/bonderlive
18           https://www.twitch.tv/shrado
19          https://www.twitch.tv/hammyie
20       https://www.twitch.tv/klondikevr
Name: What is their Twitch link, dtype: object


In [74]:
MEMBER_INDEXES = [5, 12, 19, 27]

NICKNAME_OFFSET = 0
SS_LINK_OFFSET = 2
TWITCH_LINK_OFFSET = 3

In [103]:
SS_REGEX = r'https://scoresaber\.com/u/(\d+)'

print(re.search(SS_REGEX, 'https://scoresaber.com/u/76561198799070584').group(1))

76561198799070584


In [105]:
IMGUR_REGEX = r'imgur\.com/(?:a/)?(\w+)'

print(re.search(IMGUR_REGEX, 'https://i.imgur.com/X29Xuha.png').group(1))

X29Xuha


In [99]:
TWITCH_REGEX = r'(?:https://)?(?:www.)?twitch\.tv/(\w+)/?'

print(re.search(TWITCH_REGEX, 'twitch.tv/datsurb1/about').group(1))

datsurb1


In [106]:
teams = []
player_twitch_links = {}

for key, row in df.iloc[:, :].iterrows():
  team = { 'members': [] }

  team['id'] = str(uuid.uuid4())
  team['name'] = row.iloc[1]

  imgur_image_id = re.search(IMGUR_REGEX, row.iloc[2]).group(1)
  team['logoLink'] = f'https://i.imgur.com/{imgur_image_id}.png'

  for index in MEMBER_INDEXES: 
    nickname = row.iloc[index + NICKNAME_OFFSET]

    # If no reserve present
    if str(nickname) == 'nan':
      break

    platform_id = re.search(SS_REGEX, row.iloc[index + SS_LINK_OFFSET]).group(1)
    twitch_username = re.search(TWITCH_REGEX, row.iloc[index + TWITCH_LINK_OFFSET]).group(1)
  
    player_data = requests.get(f'https://scoresaber.com/api/player/{platform_id}/basic')
    response = json.loads(player_data.text)
    country_code = response['country']

    team['members'].append({
        'nickname': nickname,
        'platformId': platform_id,
        'twitchUsername': twitch_username,
        'countryCode': country_code
    })

    player_twitch_links[platform_id] = twitch_username

  teams.append(team)

print(json.dumps(teams))
print()
print(json.dumps(player_twitch_links))

[{"members": [{"nickname": "lucazzyt", "platformId": "76561198799070584", "twitchUsername": "lucazzyt", "countryCode": "GB"}, {"nickname": "some1krille", "platformId": "76561199013335647", "twitchUsername": "some1krille", "countryCode": "DK"}, {"nickname": "Joah230", "platformId": "76561198284425720", "twitchUsername": "joah230", "countryCode": "DE"}], "id": "9fec2903-3eb1-4995-8427-d0c7770337ae", "name": "3 big baguettes", "logoLink": "https://i.imgur.com/8jcUJgb.png"}, {"members": [{"nickname": "quietkid", "platformId": "76561198425648946", "twitchUsername": "quiet_kid_15", "countryCode": "US"}, {"nickname": "Jmac", "platformId": "76561198199893013", "twitchUsername": "jmac875", "countryCode": "US"}, {"nickname": "Ayyserr", "platformId": "76561199071469065", "twitchUsername": "ayyserr24", "countryCode": "US"}, {"nickname": "BlAck_vOid", "platformId": "76561199237802861", "twitchUsername": "black_void1001", "countryCode": "US"}], "id": "aa2296c4-3a67-458c-bc77-89b1346b465a", "name": "